<a href="https://colab.research.google.com/github/shahidulislamkhokon/Python/blob/master/tweet_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import necessaries libraries

In [ ]:
!pip install transformers

In [ ]:
!pip install spacy==3.4.4

In [ ]:
# # Importing important libraris
# import pandas as pd
# import numpy as np


# # text processing libraries
# import re
# import string
# import nltk
# from nltk.corpus import stopwords

# # for plotting graphs and visualization
# from plotly.offline import iplot
# import matplotlib.pyplot as plt
# import plotly.graph_objs as go
# import cufflinks
# cufflinks.go_offline()
# cufflinks.set_config_file(world_readable=True, theme='pearl')
# import seaborn as sns


# # File system manangement
# import os

# from wordcloud import WordCloud

# # sklearn 
# from sklearn import model_selection
# from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# # Pytorch
# import torch
# import torch.nn as nn

# from tqdm import tqdm
# import string

# #Transformers
# import transformers
# import tokenizers
# from transformers import BertTokenizer, BertModel
# from transformers import AdamW
# from transformers import get_linear_schedule_with_warmup

# # Suppress warnings 
# import warnings
# warnings.filterwarnings('ignore')

In [ ]:
import re
import string
import numpy as np 
import random
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
from collections import Counter

from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator


import nltk
from nltk.corpus import stopwords

from tqdm import tqdm
import os
import nltk
import spacy
import random
from spacy.util import compounding
from spacy.util import minibatch

import warnings
warnings.filterwarnings("ignore")

import os



In [ ]:
# for using different color
def random_colours(number_of_colors):
    '''
    Simple function for random colours generation.
    Input:
        number_of_colors - integer value indicating the number of colours which are going to be generated.
    Output:
        Color in the following format: ['#E86DA4'] .
    '''
    colors = []
    for i in range(number_of_colors):
        colors.append("#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)]))
    return colors

Load Data

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/Master's_thesis/tweet_sentiment_analysis/sentiment_analysis/train.csv")
test = pd.read_csv("/content/drive/MyDrive/Master's_thesis/tweet_sentiment_analysis/sentiment_analysis/test.csv")
#ss = pd.read_csv("/content/drive/MyDrive/Master's_thesis/tweet_sentiment_analysis/sentiment_analysis/test.csv")
ss = pd.read_csv("/content/drive/MyDrive/Master's_thesis/tweet_sentiment_analysis/sentiment_analysis/submission.csv")

print(f"Training data shape: {train.shape}")
print(f"Test data shape: {test.shape}")

In [ ]:
train.info()

In [ ]:
train.dropna(inplace=True)

In [ ]:
test.info()

In [ ]:
# Checking the first few rows
train.head()

In [ ]:
train.describe()

Lets look at the distribution of tweets in the train set

In [ ]:
temp = train.groupby('sentiment').count()['text'].reset_index().sort_values(by='text',ascending=False)
temp.style.background_gradient(cmap='Purples')

In [ ]:
plt.figure(figsize=(12,6))
sns.countplot(x='sentiment',data=train)

Let's draw a Funnel-Chart for better visualization

In [ ]:
fig = go.Figure(go.Funnelarea(
    text =temp.sentiment,
    values = temp.text,
    title = {"position": "top center", "text": "Funnel-Chart of Sentiment Distribution"}
    ))
fig.show()

Generating Meta-Features

In [ ]:
def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [ ]:
results_jaccard=[]

for ind,row in train.iterrows():
    sentence1 = row.text
    sentence2 = row.selected_text

    jaccard_score = jaccard(sentence1,sentence2)
    #print(jaccard_score)
    results_jaccard.append([sentence1,sentence2,jaccard_score])
print("Results of jaccard: ", results_jaccard)    

In [ ]:
jaccard = pd.DataFrame(results_jaccard,columns=["text","selected_text","jaccard_score"])
display(jaccard)
train = train.merge(jaccard,how='outer')
display(train)


In [ ]:
train['Num_words_ST'] = train['selected_text'].apply(lambda x:len(str(x).split())) #Number Of words in Selected Text
train['Num_word_text'] = train['text'].apply(lambda x:len(str(x).split())) #Number Of words in main text
train['difference_in_words'] = train['Num_word_text'] - train['Num_words_ST'] #Difference in Number of words text and Selected Text

In [ ]:
train.head(100)

Let's look at the distribution of Meta-Features

In [ ]:
hist_data = [train['Num_words_ST'],train['Num_word_text']]

group_labels = ['Selected_Text', 'Text']

# Create distplot with custom bin_size
fig = ff.create_distplot(hist_data, group_labels,show_curve=False)
fig.update_layout(title_text='Distribution of Number Of words')
fig.update_layout(
    autosize=False,
    width=900,
    height=700,
    paper_bgcolor="LightSteelBlue",
)
fig.show()

In [ ]:
plt.figure(figsize=(12,6))
p1=sns.kdeplot(train['Num_words_ST'], shade=True, color="r").set_title('Kernel Distribution of Number Of words')
p1=sns.kdeplot(train['Num_word_text'], shade=True, color="b")

In [ ]:
plt.figure(figsize=(12,6))
p1=sns.kdeplot(train[train['sentiment']=='positive']['difference_in_words'], shade=True, color="b").set_title('Kernel Distribution of Difference in Number Of words')
p2=sns.kdeplot(train[train['sentiment']=='negative']['difference_in_words'], shade=True, color="r")

In [ ]:
plt.figure(figsize=(12,6))
sns.distplot(train[train['sentiment']=='neutral']['difference_in_words'],kde=False)

In [ ]:
plt.figure(figsize=(12,6))
p1=sns.kdeplot(train[train['sentiment']=='positive']['jaccard_score'], shade=True, color="b").set_title('KDE of Jaccard Scores across different Sentiments')
p2=sns.kdeplot(train[train['sentiment']=='negative']['jaccard_score'], shade=True, color="r")
#p2=sns.kdeplot(train[train['sentiment']=='neutral']['jaccard_score'], shade=True, color="g")
plt.legend(labels=['positive','negative'])

In [ ]:
k = train[train['Num_word_text']<=2]

In [ ]:
k.groupby('sentiment').mean()['jaccard_score']

In [ ]:
k[k['sentiment']=='positive']

Cleaning the Corpus

In [ ]:
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [ ]:
train['text'] = train['text'].apply(lambda x:clean_text(x))
train['selected_text'] = train['selected_text'].apply(lambda x:clean_text(x))

In [ ]:
train.head()

Most Common words in our Target-Selected Text

In [ ]:
train['temp_list'] = train['selected_text'].apply(lambda x:str(x).split())
top = Counter([item for sublist in train['temp_list'] for item in sublist])
temp = pd.DataFrame(top.most_common(20))
temp.columns = ['Common_words','count']
temp.style.background_gradient(cmap='Blues')

In [ ]:
fig = px.bar(temp, x="count", y="Common_words", title='Commmon Words in Selected Text', orientation='h', 
             width=700, height=700,color='Common_words')
fig.show()

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords.words('english')
def remove_stopword(x):
    return [y for y in x if y not in stopwords.words('english')]
train['temp_list'] = train['temp_list'].apply(lambda x:remove_stopword(x))

In [ ]:
top = Counter([item for sublist in train['temp_list'] for item in sublist])
temp = pd.DataFrame(top.most_common(20))
temp = temp.iloc[1:,:]
temp.columns = ['Common_words','count']
temp.style.background_gradient(cmap='Purples')

In [ ]:
fig = px.treemap(temp, path=['Common_words'], values='count',title='Tree of Most Common Words')
fig.show()

Most common word in text

In [ ]:
train['temp_list1'] = train['text'].apply(lambda x:str(x).split()) #List of words in every row for text
train['temp_list1'] = train['temp_list1'].apply(lambda x:remove_stopword(x)) #Removing Stopwords
display(train)

In [ ]:
top = Counter([item for sublist in train['temp_list1'] for item in sublist])
temp = pd.DataFrame(top.most_common(25))
temp = temp.iloc[1:,:]
temp.columns = ['Common_words','count']
temp.style.background_gradient(cmap='Blues')

In [ ]:
fig = px.bar(temp, x="count", y="Common_words", title='Commmon Words in Text', orientation='h', 
             width=700, height=700,color='Common_words')
fig.show()

Most common words Sentiments Wise

In [ ]:
Positive_sent = train[train['sentiment']=='positive']
Negative_sent = train[train['sentiment']=='negative']
Neutral_sent = train[train['sentiment']=='neutral']

In [ ]:
# MosT common positive words
top = Counter([item for sublist in Positive_sent['temp_list'] for item in sublist])
# print(top)
temp_positive = pd.DataFrame(top.most_common(10))
temp_positive.columns = ['Common_positive_words','count']
temp_positive.style.background_gradient(cmap='Greens')

In [ ]:
# MosT common Negative_sent words
top = Counter([item for sublist in Negative_sent['temp_list'] for item in sublist])
print(len(top))
temp_negative = pd.DataFrame(top.most_common(10))
temp_negative.columns = ['Common_negative_words','count']
temp_negative.style.background_gradient(cmap='Reds')

In [ ]:
# MosT common Neutral_sent words
top = Counter([item for sublist in Neutral_sent['temp_list'] for item in sublist])
# print(len(top))
temp_neutral = pd.DataFrame(top.most_common(10))
temp_neutral.columns = ['Common_Neutral_words','count']
temp_neutral.style.background_gradient(cmap='Blues')

In [ ]:
fig = px.treemap(temp_neutral, path=['Common_Neutral_words'], values='count',title='Tree Of Common_Neutral_words')
fig.show()

In [ ]:
# item = 0
# for sublist in Positive_sent['temp_list']:
#   print("sublist: ",sublist)
#   for item in sublist:
#     print('item: ',item)
#     top = Counter([item])
#     print("top: ",top)
               
# # print(item)              
               
               
# # for item in sublist])
# positive = 'positive'
# allother = []
# for item in train[train.positive != positive]['temp_list1']:
#   print("item",item)
#   for word in item:
#       allother .append(word)
# allother  = list(set(allother ))

Let's Look at Unique Words in each Segment

In [ ]:
raw_text = [word for word_list in train['temp_list1'] for word in word_list]

In [ ]:
def words_unique(sentiment,numwords,raw_words):
    '''
    Input:
        segment - Segment category (ex. 'Neutral');
        numwords - how many specific words do you want to see in the final result; 
        raw_words - list  for item in train_data[train_data.segments == segments]['temp_list1']:
    Output: 
        dataframe giving information about the name of the specific ingredient and how many times it occurs in the chosen cuisine (in descending order based on their counts)..

    '''
    allother = []
    for item in train[train.sentiment != sentiment]['temp_list1']:
      # print("item",item)
      # break
      for word in item:
          allother .append(word)
    allother  = list(set(allother ))
    # print('allother: ',allother)
    # print(len(allother))
    
    specificationally = [x for x in raw_text if x not in allother]
    # print("specificnonly",specificationally)
    # print(len(specificationally))
    
    mycounter = Counter()
    
    for item in train[train.sentiment == sentiment]['temp_list1']:
        for word in item:
            mycounter[word] += 1
    keep = list(specificationally)
    
    for word in list(mycounter):
        if word not in keep:
            del mycounter[word]
    
    Unique_words = pd.DataFrame(mycounter.most_common(numwords), columns = ['words','count'])
    
    return Unique_words

In [ ]:
Unique_Positive= words_unique('positive', 20, raw_text)
print("The top 20 unique words in Positive Tweets are:")
Unique_Positive.style.background_gradient(cmap='Greens')

In [ ]:
from palettable.colorbrewer.qualitative import Pastel1_7
plt.figure(figsize=(16,10))
my_circle=plt.Circle((0,0), 0.7, color='white')
plt.pie(Unique_Positive['count'], labels=Unique_Positive.words, colors=Pastel1_7.hex_colors)
p=plt.gcf()
p.gca().add_artist(my_circle)
plt.title('DoNut Plot Of Unique Positive Words')
plt.show()

In [ ]:
Unique_Negative= words_unique('negative', 10, raw_text)
print("The top 10 unique words in Negative Tweets are:")
Unique_Negative.style.background_gradient(cmap='Reds')

In [ ]:
Unique_Neutral= words_unique('neutral', 10, raw_text)
print("The top 10 unique words in Neutral Tweets are:")
Unique_Neutral.style.background_gradient(cmap='Oranges')

In [ ]:
from palettable.colorbrewer.qualitative import Pastel1_7
plt.figure(figsize=(16,10))
my_circle=plt.Circle((0,0), 0.7, color='white')
plt.pie(Unique_Neutral['count'], labels=Unique_Neutral.words, colors=Pastel1_7.hex_colors)
p=plt.gcf()
p.gca().add_artist(my_circle)
plt.title('DoNut Plot Of Unique Neutral Words')
plt.show()

It's Time For WordClouds

In [ ]:
def plot_wordcloud(text, mask=None, max_words=200, max_font_size=100, figure_size=(24.0,16.0), color = 'white',
                   title = None, title_size=40, image_color=False):
    stopwords = set(STOPWORDS)
    more_stopwords = {'u', "im"}
    stopwords = stopwords.union(more_stopwords)

    wordcloud = WordCloud(background_color=color,
                    stopwords = stopwords,
                    max_words = max_words,
                    max_font_size = max_font_size, 
                    random_state = 42,
                    width=400, 
                    height=200,
                    mask = mask)
    wordcloud.generate(str(text))
    
    plt.figure(figsize=figure_size)
    if image_color:
        image_colors = ImageColorGenerator(mask);
        plt.imshow(wordcloud.recolor(color_func=image_colors), interpolation="bilinear");
        plt.title(title, fontdict={'size': title_size,  
                                  'verticalalignment': 'bottom'})
    else:
        plt.imshow(wordcloud);
        plt.title(title, fontdict={'size': title_size, 'color': 'black', 
                                  'verticalalignment': 'bottom'})
    plt.axis('off');
    plt.tight_layout() 
d = "/content/drive/MyDrive/Master's_thesis/tweet_sentiment_analysis/sentiment_analysis/"    

In [ ]:
pos_mask = np.array(Image.open(d+ '0x0.jpg'))
plot_wordcloud(Neutral_sent.text,mask=pos_mask,color='white',max_font_size=100,title_size=30,title="WordCloud of Neutral Tweets")

In [ ]:
plot_wordcloud(Positive_sent.text,mask=pos_mask,title="Word Cloud Of Positive tweets",title_size=30)

In [ ]:
plot_wordcloud(Negative_sent.text,mask=pos_mask,title="Word Cloud of Negative Tweets",color='white',title_size=30)

Modelling

1)Modelling the Problem as NER

In [ ]:
df_train = pd.read_csv("/content/drive/MyDrive/Master's_thesis/tweet_sentiment_analysis/sentiment_analysis/train.csv")
df_test = pd.read_csv("/content/drive/MyDrive/Master's_thesis/tweet_sentiment_analysis/sentiment_analysis/test.csv")
df_submission = pd.read_csv("/content/drive/MyDrive/Master's_thesis/tweet_sentiment_analysis/sentiment_analysis/submission.csv")

In [ ]:
df_train['Num_words_text'] = df_train['text'].apply(lambda x:len(str(x).split())) #Number Of words in main Text in train set

In [ ]:
df_train = df_train[df_train['Num_words_text']>=3]

In [ ]:
def save_model(output_dir, nlp, new_model_name):
    ''' This Function Saves model to 
    given output directory'''
    
    output_dir = f"/content/drive/MyDrive/Master's_thesis/tweet_sentiment_analysis/sentiment_analysis/{output_dir}"
    if output_dir is not None:        
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        nlp.meta["name"] = new_model_name
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

In [ ]:
# pass model = nlp if you want to train on top of existing model 
from spacy.training.example import Example

def train(train_data, output_dir, n_iter, model=None):
    """Load the model, set up the pipeline and train the entity recognizer.
    remove nlp.create_pipe and call `nlp.add_pipe('name')` instead.
    """
    if model is not None:
        nlp = spacy.load(output_dir)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")  # create blank Language class
        # print("nlp",nlp)
        print("Created blank 'en' model")
    
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "ner" not in nlp.pipe_names:
        # ner = nlp.create_pipe("ner")
        ner = nlp.add_pipe('ner', last=True)
        # # nlp.add_pipe('ner')
        # ner = nlp.add_pipe('ner')
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe("ner")
    
    # add labels
    for _, annotations in train_data:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])
            # print("ent", ent)
    # print("this is done")
    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.disable_pipes(*other_pipes):  # only train NER
        # sizes = compounding(1.0, 4.0, 1.001)
        # batch up the examples using spaCy's minibatch
        if model is None:
            nlp.begin_training()
        else:
            nlp.resume_training()
        # print("this is done 2")

        # for batch in spacy.util.minibatch(TRAINING_DATA, size=2):
        #     for text, annotations in batch:
        #       # create Example
        #       doc = nlp.make_doc(text)
        #       example = Example.from_dict(doc, annotations)
        #       # Update the model
        #       nlp.update([example], losses=losses, drop=0.3)


        for itn in tqdm(range(n_iter)):
          # print("this is done 3")
          random.shuffle(train_data)
          batches = minibatch(train_data, size=compounding(4.0, 500.0, 1.001))    
          losses = {}
          # print("this is done 4")

          # for text, annotations in batches:
          #   # create Example
          #   doc = nlp.make_doc(text)
          #   example = Example.from_dict(doc, annotations)
          #   # Update the model
          #   nlp.update([example], losses=losses, drop=0.3)
          # for batch in batches:
          #   print("this is done 5")
          #   texts, annotations = zip(*batch)
          #   nlp.update(texts,  # batch of texts
          #               annotations,  # batch of annotations
          #               drop=0.5,   # dropout - make it harder to memorise data
          #               losses=losses, 
          #               )

          for batch in batches:
            texts, annotations = zip(*batch)
            # print("texts",texts)
            
            example = []
            # Update the model with iterating each text
            for i in range(len(texts)):
                doc = nlp.make_doc(texts[i])
                # print("doc",doc)
                example.append(Example.from_dict(doc, annotations[i]))
                # print("example",example)
            
            # Update the model
            nlp.update(example, drop=0.5, losses=losses)

        print("Losses", losses)
    save_model(output_dir, nlp, 'st_ner')

In [ ]:
def get_model_out_path(sentiment):
    '''
    Returns Model output path
    '''
    model_out_path = None
    if sentiment == 'positive':
        model_out_path = "/content/drive/MyDrive/Master's_thesis/tweet_sentiment_analysis/sentiment_analysis/archive/models/model_pos"
    elif sentiment == 'negative':
        model_out_path = "/content/drive/MyDrive/Master's_thesis/tweet_sentiment_analysis/sentiment_analysis/archive/models/model_neg"
    print("get_model_out_path is done")
    return model_out_path


In [ ]:
def get_training_data(sentiment):
    '''
    Returns Trainong data in the format needed to train spacy NER
    '''
    train_data = []
    for index, row in df_train.iterrows():
        if row.sentiment == sentiment:
            selected_text = row.selected_text
            text = row.text
            start = text.find(selected_text)
            end = start + len(selected_text)
            train_data.append((text, {"entities": [[start, end, 'selected_text']]}))
    print("get_training_data data is done")        
    return train_data

In [ ]:
sentiment = 'positive'
train_data = get_training_data(sentiment)
model_path = get_model_out_path(sentiment)
# For DEmo Purposes I have taken 3 iterations you can train the model as you want
train(train_data, model_path, n_iter=3, model=None)

In [ ]:
sentiment = 'negative'

train_data = get_training_data(sentiment)
model_path = get_model_out_path(sentiment)

train(train_data, model_path, n_iter=3, model=None)

Predicting with the trained Model

In [ ]:
def predict_entities(text, model):
    doc = model(text)
    ent_array = []
    for ent in doc.ents:
        start = text.find(ent.text)
        end = start + len(ent.text)
        new_int = [start, end, ent.label_]
        if new_int not in ent_array:
            ent_array.append([start, end, ent.label_])
    selected_text = text[ent_array[0][0]: ent_array[0][1]] if len(ent_array) > 0 else text
    return selected_text

In [ ]:
selected_texts = []
MODELS_BASE_PATH = "/content/drive/MyDrive/Master's_thesis/tweet_sentiment_analysis/sentiment_analysis/content/drive/MyDrive/Master's_thesis/tweet_sentiment_analysis/sentiment_analysis/archive/models/"

if MODELS_BASE_PATH is not None:
    print("Loading Models  from ", MODELS_BASE_PATH)
    model_pos = spacy.load(MODELS_BASE_PATH + 'model_pos')
    model_neg = spacy.load(MODELS_BASE_PATH + 'model_neg')
        
    for index, row in df_test.iterrows():
        text = row.text
        output_str = ""
        if row.sentiment == 'neutral' or len(text.split()) <= 2:
            selected_texts.append(text)
        elif row.sentiment == 'positive':
            selected_texts.append(predict_entities(text, model_pos))
        else:
            selected_texts.append(predict_entities(text, model_neg))
        
df_test['selected_text'] = selected_texts


In [ ]:
df_submission['selected_text'] = df_test['selected_text']
df_submission.to_csv("submission.csv", index=False)
display(df_submission.head(10))